# Preprocessing and train/test split based only on the toxic label

## Quick look at the shape of the data

In [ ]:
import pandas as pd
pd.set_option("display.max_columns", 100)
pd.set_option("display.max_rows", 100)
pd.set_option("display.max_colwidth", -1)

In [ ]:
data = pd.read_csv("./data/raw-data/train.csv")

In [ ]:
data.shape

In [ ]:
data.describe()

## Split to train and test

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit

split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=543553)

In [ ]:
trainIndex, testIndex = list(split.split(data, data.toxic))[0]

In [ ]:
train, test = data.iloc[trainIndex], data.iloc[testIndex]

## Filter punctuation

In [ ]:
def filterColumnPunctuation(column):
    return (column
    .str.replace("[^A-Za-z\s0-9]", "")
    .str.replace("\s", " ")
    .str.lower())

def filterPunctuation(dataset, column):
    return pd.concat([
            dataset.drop([column], axis=1),
            filterColumnPunctuation(dataset[column])], 
        axis=1)

In [ ]:
%%time
trainFiltered, testFiltered, dataFiltered = list(map(
    lambda dataset: filterPunctuation(dataset, "comment_text"), 
    [train, test, data]))

In [ ]:
trainFiltered.head()

In [ ]:
testFiltered.head()

In [ ]:
dataFiltered.head()

In [ ]:
trainFiltered.shape

In [ ]:
testFiltered.shape

In [ ]:
dataFiltered.shape

## Export data

In [ ]:
from os import path
import csv

def exportDataset(dataset, filename, exportDirectory):
    dataset.to_csv(
        path.join(exportDirectory, filename), 
        index=False, 
        escapechar="\\",
        quoting=csv.QUOTE_NONNUMERIC)

In [ ]:
exportDirectory = "./data/preprocessed-train-test"
datasetFilenames = zip(
    [trainFiltered, testFiltered, dataFiltered], 
    ["train.csv", "test.csv", "all.csv"])

for dataset, filename in datasetFilenames:
    exportDataset(dataset, filename, exportDirectory)